Basic Dashboard
   - User needs to provide their key, modify database host if necessary
   
This is a readonly notebook, after configuring run all cells and review the results.

Differences from Qualitative:
  - Data presented on a scale from 0 - 100
  - Pervasive attribute
  - Impact Category attribute
  - Estimated number of attacks
  - Estimated Annual Loss Expectancy

In [ ]:
userkey = "140"
dbhost  = 'localhost'

In [ ]:
import pandas as pd
import ipywidgets as widgets
import psycopg2
import qual

conn = psycopg2.connect("postgres://csc452:csc452@" + dbhost + "/csc452")  
cursor = conn.cursor()  
cursor.execute('SET mydomain.userkey = ' + userkey)

triages = pd.read_sql_query ('SELECT name, description, vulnerability, pervasive, impact, impact_name, impact_description, source_name, source_description, capacity, init FROM triagev', conn)


for i in triages.index:
    v = qual.qdict[qual.norm(triages.at[i,'vulnerability'])]
    l = qual.qdict[qual.norm(triages.at[i,'init'])]
    im = qual.qdict[qual.norm(triages.at[i,'impact'])]
    o = qual.overall[l][v]
    triages.at[i,'overall_likelihood'] = o
    triages.at[i,'risk_level'] = qual.risk_level[o][im]
    triages.at[i, 'risk_num'] = qual.qval_dict[qual.risk_level[o][im]]
    triages.at[i, 'annual_events'] = triages.at[i,'init'] * qual.rate_dict[l]
    triages.at[i, 'event_loss'] = triages.at[i,'impact'] * qual.loss_dict[im]
    triages.at[i, 'annual_loss'] = triages.at[i,'annual_events'] * triages.at[i,'event_loss']

# still want to calculate the qualitative risk assessments
triages = triages.set_index ('name')
tr = triages.loc[triages['source_name']!='source name']

tr.sort_values (by=['annual_loss'])

print ("Threat Overview")
tr.filter (items=['name', 'annual_loss', 'risk_level', 'overall_likelihood', 'annual_events', 'event_loss', 'description'])


In [ ]:
print ("Threat Sources")
tr.filter (items=['name', 'source_name', 'source_description', 'init'])

In [ ]:
print ("Threat Impacts")
tr.filter (items=['name', 'impact', 'impact_name', 'impact_description', 'pervasive', 'vulnerability'])